#  CDC Data Exploration

## The Data
We are exploring the linked birth/infant death data from the CDC. The data we are using is cohort data from 2008. More on what cohort vs period linked means later. The CDC collects live birth and infant death data from across the US and territories, does some analysis (published by national vital statistics ) and makes the data available to the public. The concept of linking is that each infant death (death within the first year of life) record is linked to the birth certificate (live birth record) if possible. It is not always possible to do this and there are some small number of death records that are not linked to corresponding births. 

The reason this linking is done is to enable researchers to determine contributing factors related to the infant death. The live birth data contains information about the subject that is not contained in the death record. This information can be used to help determine contributing factors about the cause of death. The live birth data contains information about the pregnancy such as gestational age of the infant at birth, the birth weight, medical conditions of the mother, parents ethnicity and race, etc. In all there are around four hundred characteristics assembled for each birth record. Each year there are approximately four million births in the US. 
### Period vs Cohort
The terms numerator and denominator are encountered when dealing with percentages analysis of populations. Meaningful information about infant death is cast as being a fraction of the total live births. So for example the probability of infant death based on population would be calculated as the number of infant deaths divided by the total number of live births in a given year. The death records are the *numerator* and the live births are the *denominator*. So one talks about sets of records that make up the numerator and denominator in a particular study. The denominator for our work consists of the records for live births occuring in 2008. 

The numerator consists of the death records. The numerator for the *period* linked data consists of all infant deaths that occured in a particular year regardless of when the birth occured. For our year this means some of the deaths in the numerator were for births occuring in 2007. Cohort linked numerator data consists of all death records for infants born in a given year. The death for some of these infants may occur in the following year. One can see there are fewer issues in linking cohort data since each death corresponds to a birth in a particular year. The denominator for both cohort an d period is the same live birth data for a particular year. We will use cohort data. 

You will not be responsible for any of the particulars of CDC data. 

### Data Source
The data is available from the [vital statistics online data portal](https://www.cdc.gov/nchs/data_access/vitalstatsonline.htm). There you will find period linked data from 1995 through 2017. The cohort linked data runs from 1995 through 2014. You will also find *user guides* for each year of data. The guide is an invaluable source of information about what is in the datasets and how the data is laid out. We will refer to this document when we construct data readers for the particular datasets. Data are added to this page on an irregular basis. The NVSS publishes yearly infant mortality statistics reports. Those reports and other related data can be found [here](https://www.cdc.gov/nchs/nvss/linked-birth.htm). This site also has links to the downloadable data and some other information about how the data are collected and processed. This is a good source of information about infant mortality in the US and territories. 

An unfortunate fact is that the CDC data is not consistent from one year to another. Some years contain dat fields (characteristics associated with each birth record such as birth weight) that other years do not. In addition great effort has been expended to be sure the data is not identifiable. That is CDC has removed information that may help someone identify a particular birth as being associated with a particular individual. While this increases privacy it makes the analysts job more difficult if, for example, the work involved regional or environmental factors. While these types of changes are reasonable, others seem arbitrary. Some fields are simply dropped or the name of the field is changed from one year to another. This makes the analysis more difficult across years. We shall only concentrate on a single year however.


## Analysis
Now that we know something about the data lets do some data exploration. First off we need to be able to read the particular CDC data format. The CDC data is contained in a large ASCII file with one record per line. To extract the data for a particular record one must know what characters in the line correspond to that particular data. For example, in the 2008 data the birt hweight in grams is located in position 466-470 of each record. Constructing a table for analysis then consists of selecting variables and reading their values for each record. Once the table of data is collected we can then process the data and extract information. 

### IO
The python required to read the records from the raw data is not complex but python IO is not the focus of this notebook. In order to make the process simpler we use an IO module from a Python package that the data group created for that purpose. Interested readers can examine that module directly. It will be available on canvas. The following code snippet sets the location of the tools package, imports the necessary tool bits and loads the pandas and matplotlib packages.

In [1]:
# Prepare the environment to access some example code and some data
import sys
import os
# add the location of the data tools to the pythonpath
# on windows I need to add the path to the CDCDataTools module 
# or this notebook needs to be in the same directory as that module
sys.path.append('C:\\Users\\Dave Semeraro\\')
#import some stuff for later. 
#from CDCDataTools import IO
from CDCDataTools import IO
import pandas as pd
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'CDCDataTools'

Now lets read some data into a datatable or dataframe as pandas calls it. This will take a couple of minutes so be patient. 

In [ ]:
# where the data is
datadir = 'C:\\Users\\Dave Semeraro\\Desktop'

NameOfBirthFile = os.path.join(datadir,'VS08LKBC.DUSDENOM')
NameOfUnlinkedDeathFile = os.path.join(datadir,'VS08LKBC.USUNMPUB')
NameOfLinkedDeathFile = os.path.join(datadir,'VS08LKBC.USNUMPUB')
# select some fields and make a list
LinkedDeathColumns   = ['RESTATUS','SEX','COMBGEST','ESTGEST','OBGEST_FLG','BRTHWGT','AGED','DOB_YY','DPLURAL','DTHYR','SEQNUM']
UnlinkedDeathColumns = ['RESTATUS','SEX','COMBGEST','ESTGEST','OBGEST_FLG','BRTHWGT','AGED','DOB_YY','DPLURAL','DTHYR']
BirthColumns         = ['RESTATUS','SEX','COMBGEST','ESTGEST','OBGEST_FLG','BRTHWGT','AGED','DOB_YY','DPLURAL','DTHYR']
# Read some data
LinkedDeaths = IO.readCDC(NameOfLinkedDeathFile,LinkedDeathColumns)
UnlinkedDeaths = IO.readCDC(NameOfUnlinkedDeathFile,UnlinkedDeathColumns)
Births = IO.readCDC(NameOfBirthFile,BirthColumns)
print('finished reading data')

The code above sets up the data read by specifying the file names and the variables to be read for each record in the respective files. There are over 400 variables per record so this is a small subset. By specifying only the columns or variables we need we can greatly reduce the amount of memory we use in the analysis. 

The readCDC function takes the file name and list of variables as input and returns the pandas dataframe with the data. It takes a couple of minutes to input the data so give the cell some time to complete. It will print 'finished reading data' when its done. Now we can move on to the interesting bits. 

### What's in the data
How big is the data? Lets count the number of records in each table. 

In [ ]:
# count the number of records and print 'em out
total_linked_deaths = len(LinkedDeaths)
total_unlinked_deaths = len(UnlinkedDeaths)
total_births = len(Births)
print('Totals')
print(f'Linked {total_linked_deaths:6} deaths')
print(f'Unlinded {total_unlinked_deaths:6} deaths')
print(f'Births {total_births:6} births')

When an infant dies in the first 4 weeks of birth it is termed *neonatal death*. Lets find out how many if the infant deaths are neonatal deaths. Lets add a column to the death tables that contains a boolean value that is true if the AGED variable is < 28 for each record. For that we will use a lambda function and the apply function of the dataframe object. 



In [ ]:
# add a column to the death dataframes
LinkedDeaths['NEONDTH'] = LinkedDeaths.apply(lambda row: int(row.AGED) < 28, axis='columns')
UnlinkedDeaths['NEONDTH'] = UnlinkedDeaths.apply(lambda row: int(row.AGED) <28, axis='columns')
total_linked_neonatal_deaths = (LinkedDeaths.NEONDTH).value_counts()[True]
total_unlinked_neonatal_deaths = (UnlinkedDeaths.NEONDTH).value_counts()[True]
print(f'Linked {total_linked_neonatal_deaths:6} neonatal deaths')
print(f'Unlinked {total_unlinked_neonatal_deaths:6} neonatal deaths')


A bit of explanation. The command LinkedDeaths.NEONDTH extracts the NEONDTH column from the dataframe in the form of a pandas series (think linear algebra vector). The value_counts() function counts the occurance of each value in the series. In our case how many True and how many False occur. We are only interested in how many True values there are so we index that result by True. 

A dataframe also has an apply function. The apply function does what the name says. It applies the function to the elements of the dataframe either across the rows or columns. We use a lambda function that takes a row of the dataframe as input and calculates the new column value based on the AGED element of that row. 

We see that of the total linked infant deaths a significant fraction are neonatal deaths. That is a meaningful result. Now lets investigate the distribution of neonatal deaths as a function of gestational age. In order to do that we need to count the number of deaths corresponding to the different gestational ages. Gestational age is measured in the number of weeks from conception to birth. The CDC record for gestationalage is COMBGEST. 


In [ ]:
# lets join the linked and unlinked deaths into one table. 
Deaths = pd.concat([LinkedDeaths,UnlinkedDeaths],ignore_index=False,join='inner')
# now lets extract the neonatal deaths from the combinded table
neondeaths = Deaths.query("NEONDTH==True")
neondeathcount = neondeaths['COMBGEST'].value_counts().sort_index()
neondeathcount

So this is odd. The output has several values that dont make sense. There are 276 values that have no gestational age and 232 that have a gestational age of 99 weeks. This is clearly wrong. Consulting the user guide tells us that a value of 99 indicates missing or unknown data. The blank data is simply missing. We need to clean this data before we can make sense of it. We need to remove the records with unknown gestational age. We can do that with another querey. Lets reformulate the neondeaths table query. 

In [ ]:
neondeaths = Deaths.query("NEONDTH==True&COMBGEST!='  '&COMBGEST!='99'")
neondeathcount = neondeaths['COMBGEST'].value_counts().sort_index()
neondeathcount

Thats better. Now lets plot the data and see what it tells us. We will use a bar chart for this data

In [ ]:
plt.figure(1)
dead = neondeathcount.plot.bar()
dead.set_title("Neonatal deaths 2008",fontsize=18)
dead.set_xlabel("gestational Age (Combgest)",fontsize=16)
dead.set_ylabel("Deaths",fontsize=16)

This figure shows us the frequency plot of neonatal death vs gestational age. It tells us most of the deaths occur for preterm infants. Preterm infants are those born before reaching 37 weeks of gestation. The plot shows that there are two peaks in neonatal deaths. The first occurs at around 22 weeks and the second peak is  a couple of weeks after term. Another interesting observation is the presence of some very long gestations. Given the average term delivery is at 38 weeks it seems unlikely that some women deliver a full two months after that. It could be true I suppose but I would not want to be that mom. 